# O Problema
Sliding Puzzle - Bloco Deslizante

In [ ]:
# !wget -qq https://miro.medium.com/max/700/1*W7jg4GmEjGBypd9WPktasQ.gif
from IPython.display import Image
Image(url='https://miro.medium.com/max/700/1*W7jg4GmEjGBypd9WPktasQ.gif',width=200)

# Resolver o quebra-cabeças usando Buscas

In [24]:
!pip install Levenshtein

     |████████████████████████████████| 163kB 7.7MB/s 


In [25]:
import numpy as np 
from collections import deque 
import itertools
import functools
import resource
import sys
import Levenshtein

In [26]:
class Node(): 
    def __init__ (self, board, parent): 
        self.board = board 
        self.signature = self.get_signature() 
        self.parent = parent
        self.empty_pos = self.get_empty_pos() 
        self.cost = 0
        self.children = []

    def get_signature (self): 
        flattenBoard = list(itertools.chain.from_iterable(self.board)) 
        listToSignature = lambda a, b: str(a) + str(b)

        return functools.reduce(listToSignature, flattenBoard)

    def get_possible_moves (self): 
        all_moves = [
            (0, 1),
            (1, 0),
            (0, -1), 
            (-1, 0)
            ] 
        
        x, y = self.empty_pos 
        possibleMoves = list() 

        for xDelta, yDelta in all_moves: 
            newX = x + xDelta
            newY = y + yDelta 
            
            if (-1 < newX < 3) and ((-1 < newY < 3)):
                move = (newX, newY) 
                possibleMoves.append(move) 
        
        return possibleMoves 

 
    def apply_move (self, move): 
        new_board = self.board.copy()

        new_board[self.empty_pos], new_board[move] = new_board[move], new_board[self.empty_pos]
        return new_board

    def get_children (self): 
        children = [] 
        possible_moves = self.get_possible_moves() 

        for move in possible_moves: 
            child_board = self.apply_move(move)  
            child = Node(child_board, self)

            children.append(child) 

        self.children = children 

    def get_empty_pos (self): 
        y, x = np.where(self.board == 0)
        x = x.item() 
        y = y.item()
     
        return (y, x)

In [27]:
def print_solution (initial_node, end_node): 
  solution = []
  while not (end_node.parent == None): 
    solution.append(end_node.board)
    end_node = end_node.parent 
  
  solution = solution[::-1]

  print(initial_node.board)
  print("")

  for board in solution:
    print(board)
    print("")


In [56]:
def h(node): 
  # Uses Levenshteins distance as heuristic
  solution_signature = "123456780"
  return Levenshtein.distance(node.signature, solution_signature)

In [ ]:
def h(node): 
  # Uses tiles misplaced as heuristic
  solution_signature = "123456780"
  cost = 0
  for i, j in zip(solution_signature, node.signature): 
    if i =! j: 
      cost += 1
  return cost 

In [68]:
def h(node): 
  # Uses gradient as heuristic
  solution_signature = "123456780"
  cost = 0
  for i, j in zip(solution_signature, node.signature):  
    cost += abs(int(i) - int(j)) 
  return cost 

In [83]:
def h(node): 
  # Uses distance from Gauss sum as heuristic 
  gauss_sum = 9
  cost = 0 

  for i, j in zip(range(4), range(-2, -6, -1)): 
   cost += abs(int(node.signature[i]) + int(node.signature[j]) - gauss_sum) 
  return cost

## A*

In [84]:
def a_star(initial_node):
    solution_signature = "123456780"
    visited_nodes = set() 

    if initial_node.signature == solution_signature:
        return initial_node
  
    q = deque()
    q.append(initial_node)
    visited_nodes.add(initial_node)

    while (len(q) > 0):
        q = deque(sorted(list(q), key=lambda node: node.cost))
        q[0].get_children()

        for i in q[0].children:
            if i in visited_nodes: 
              continue
            i.cost = h(i) + len(q)**1.5 + i.parent.cost
            visited_nodes.add(i)

            if i.signature == solution_signature:
                return i
            q.append(i)
        q.popleft()

In [91]:
board =np.array([[2, 3, 5],
                 [1, 4, 0],
                 [7, 8, 6]])

In [92]:
initial_node = Node(board=board, parent=None)
end_node = a_star(initial_node)

In [93]:
print_solution(initial_node, end_node)

[[2 3 5]
 [1 4 0]
 [7 8 6]]

[[2 3 0]
 [1 4 5]
 [7 8 6]]

[[2 0 3]
 [1 4 5]
 [7 8 6]]

[[0 2 3]
 [1 4 5]
 [7 8 6]]

[[1 2 3]
 [0 4 5]
 [7 8 6]]

[[1 2 3]
 [4 0 5]
 [7 8 6]]

[[1 2 3]
 [4 5 0]
 [7 8 6]]

[[1 2 3]
 [4 5 6]
 [7 8 0]]



## Considerações

 &nbsp;&nbsp;&nbsp;&nbsp;  A escolha da heurística é uma tarefa complexa e as alternativas são muitas. Neste problema é dificil determinar uma heurística simples, visto que a "ordenação" das peças é importante, mas as soluções mais otimizadas, as vezes, podem ser bastante "desordenadas". <br>
&nbsp;&nbsp;&nbsp;&nbsp;  Uma heurística possível, que não foi testada, é a de uma janela deslizante bidimensional(2x2) na matriz do jogo verificando se os números próximos aquele setor estão contidos, dado que este é um método utilizado por jogadores reais. <br><br>
&nbsp;&nbsp;&nbsp;&nbsp; Quanto à consumo de memória, o algorítmo pode expandir e vir a visitar todos os elementos possíveis, porém na média se sai muito bem. Já ao processamento, o calculo de custo e ordenação da fila de estados com prioridade incrementam a complexidade de tempo.
